In [83]:
#files to open: train and test data,

from google.colab import files

uploaded = files.upload()


Saving test_v7_v8.csv to test_v7_v8.csv
Saving test_v3.csv to test_v3.csv


In [0]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.utils import class_weight
from sklearn.metrics import recall_score, precision_score, confusion_matrix

In [0]:
import numpy as np
from keras.models import Model
from keras.layers import Dense, Input, Dropout, LSTM, Activation

In [0]:
import matplotlib.pyplot as plt
%matplotlib inline

def plot_loss(epochs, loss, val_loss):
    plt.figure(figsize=(10, 7))
    plt.plot(np.arange(epochs), loss, label='loss')
    plt.plot(np.arange(epochs), val_loss, label='val_loss')
    plt.legend()
    plt.grid(True)
def acc_graph(epochs, train_acc, val_acc):
    x_axis = np.linspace(1, epochs, num=epochs)
    
    plt.figure(figsize=(10, 7))
    plt.plot(x_axis, train_acc, color='g', lw=3, alpha=0.7, label='Train Accuracy')
    plt.plot(x_axis, val_acc, color='orange', lw=3, alpha=0.7, label='Val Accuracy')
    plt.title('Accuracy graph')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend(loc='lower right')
    plt.grid(True)

In [0]:
def calculation_classes(y, y_test = None):
    print ('\t\tTRAIN DATA\tTEST DATA')
    print ('Negative\t', np.sum(y==0),'  ',
           round(100*np.sum(y==0)/y.shape[0],2),'%',
           '\t',np.sum(y_test==0),'  ',
           round(100*np.sum(y_test==0)/y_test.shape[0],2),'%'
           '\nPositive\t ', np.sum(y==1),' ',
           round(100*np.sum(y==1)/y.shape[0],2),'%'
           '\t',np.sum(y_test==1),' ',
           round(100*np.sum(y_test==1)/y_test.shape[0],2),'%'
      )

In [0]:
#from utils import model

In [0]:
def make_predictions(X, y, out=False, model=GRU_model, X_ini=X_ini):
  preds = model.predict(X)
  y_predict = np.where(preds > 0.5, 1, 0)
  recall = recall_score(y, y_predict)
  print('Recall = ', recall, 'Precision = ', precision_score(y, y_predict))
  cm  = confusion_matrix(y, y_predict)
  print(cm)
  if out is True:
    for i in range(X.shape[0]):
      if abs(y_predict[i] - y[i])!= 0:
        print(y[i], preds[i], X_ini[i], np.ndarray.tolist(X[i]))
    

# Import data sets

In [0]:
#data_train = pd.read_csv('sources/train.csv')
#data_test = pd.read_csv('sources/test_v7_v8.csv')

data_train = pd.read_csv('train.csv')


In [0]:
data_test = pd.read_csv('test_v7_v8.csv')


In [0]:
#data_test = pd.read_csv('sources/test_v3.csv')
data_test2 = pd.read_csv('test_v3.csv')


In [0]:
X = data_train.drop(['Y'], axis='columns').values
y = data_train['Y'].values
#print(y)
X_ini = data_train.drop(['Y'], axis='columns').values

In [0]:
X_test = data_test.drop(['Y'], axis='columns').values
test =  data_test.drop(['Y'], axis='columns').values
y_test = data_test['Y'].values

X_test2 = data_test2.drop(['Y'], axis='columns').values
test2 =  data_test2.drop(['Y'], axis='columns').values
y_test2 = data_test2['Y'].values


In [0]:
m = X.shape[0]
X_mean = np.zeros((m,1))
for i in range(m):
  X_mean[i] = np.mean(X[i,:], axis=0)

In [0]:
ss = StandardScaler()
ss.fit(X_mean)

X = ss.transform(X)

X_test = ss.transform(X_test)
X_test2 = ss.transform(X_test2)

In [0]:
#print(ss.mean_, ss.var_)
#print(X_test)

In [145]:
X_train,X_dev,y_train,y_dev = train_test_split(X, y, test_size=0.2, \
                                               shuffle=True, stratify=y)

class_weights_train = class_weight.compute_class_weight('balanced',
                                                 np.unique(y_train),
                                                 y_train)

class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(y), y)
print(class_weights)
print(class_weights[0]/(class_weights[1]+class_weights[0]))


[ 0.51310078 19.5828305 ]
0.02553257048418045


In [151]:
print(X_train[0], X_dev[0])

[[ 0.17556581]
 [ 0.07228878]
 [-0.11533115]
 [ 0.11876345]
 [ 0.0275354 ]
 [-0.07057777]] [[ 0.12564858]
 [ 0.16179554]
 [ 0.14286142]
 [ 0.0705675 ]
 [-0.12221629]
 [ 0.16351683]]


In [0]:
#calculation_classes(y, y_dev)
#calculation_classes(y_train, y_dev)

In [146]:
X = np.expand_dims(X, axis=2)
X_train = np.expand_dims(X_train, axis=2)
X_dev = np.expand_dims(X_dev, axis=2)

print(X_test.shape)
X_test = np.expand_dims(X_test, axis=2)
X_test2 = np.expand_dims(X_test2, axis=2)
print(X_test.shape)

(59, 6)
(59, 6, 1)


In [147]:
print(X.shape)

(71399, 6, 1)


# NN on train/dev sets

In [0]:
from keras.models import Model, Sequential
from keras.layers import Dense, Input, GRU, LSTM
from keras.optimizers import Adam
from keras.callbacks import LearningRateScheduler


def model_GRU(input_shape, Ty, hidden_size):
    i = Input(shape=input_shape, dtype='float32')
    X = GRU(hidden_size, return_sequences=False)(i)
    
    X = Dense(Ty, activation='sigmoid')(X)
    model = Model(inputs=[i], outputs=X)
    
    return model
  
def model_LSTM(hidden_size):  
  model = Sequential()
  model.add(LSTM(hidden_size, return_sequences=False))
  model.add(Dense(units=1, activation='sigmoid'))
  
  return model


In [0]:
# For ready model
GRU_model = model_GRU((Tx,1),Ty)#LSTM_model.summary()
GRU_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


GRU_model.load_weights('GRU_sequence_weights.h5')

In [0]:
Tx=  6
Ty = 1
epochs = 20
batch_size = 32

In [59]:
GRU_model = model_LSTM(16)
opt = Adam(lr=0.01, beta_1=0.9, beta_2=0.999)
GRU_model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
logs = GRU_model.fit(X, y , epochs=epochs, batch_size=batch_size, shuffle=True,\
                    validation_data=(X_dev, y_dev), verbose=2, class_weight=class_weights_train, \
                    initial_epoch=0)

Train on 71399 samples, validate on 14280 samples
Epoch 1/10
 - 9s - loss: 0.0424 - acc: 0.9871 - val_loss: 0.0317 - val_acc: 0.9888
Epoch 2/10
 - 8s - loss: 0.0335 - acc: 0.9883 - val_loss: 0.0314 - val_acc: 0.9904
Epoch 3/10
 - 8s - loss: 0.0323 - acc: 0.9882 - val_loss: 0.0312 - val_acc: 0.9879
Epoch 4/10
 - 8s - loss: 0.0259 - acc: 0.9894 - val_loss: 0.0203 - val_acc: 0.9905
Epoch 5/10
 - 8s - loss: 0.0190 - acc: 0.9906 - val_loss: 0.0198 - val_acc: 0.9863
Epoch 6/10
 - 8s - loss: 0.0178 - acc: 0.9911 - val_loss: 0.0164 - val_acc: 0.9908
Epoch 7/10
 - 8s - loss: 0.0166 - acc: 0.9919 - val_loss: 0.0149 - val_acc: 0.9919
Epoch 8/10
 - 8s - loss: 0.0160 - acc: 0.9921 - val_loss: 0.0155 - val_acc: 0.9924
Epoch 9/10
 - 8s - loss: 0.0160 - acc: 0.9922 - val_loss: 0.0167 - val_acc: 0.9888
Epoch 10/10
 - 8s - loss: 0.0158 - acc: 0.9919 - val_loss: 0.0139 - val_acc: 0.9936


In [0]:
opt = Adam(lr=0.001, beta_1=0.9, beta_2=0.999)
GRU_model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])



In [156]:
n_epochs = 20
logs = GRU_model.fit(X_train, y_train , epochs=n_epochs, batch_size=batch_size, shuffle=True,\
                    validation_data=(X_dev, y_dev), verbose=2, class_weight=class_weights_train)

Train on 57119 samples, validate on 14280 samples
Epoch 1/20
 - 11s - loss: 0.0063 - acc: 0.9977 - val_loss: 0.0058 - val_acc: 0.9976
Epoch 2/20
 - 7s - loss: 0.0063 - acc: 0.9977 - val_loss: 0.0062 - val_acc: 0.9975
Epoch 3/20
 - 7s - loss: 0.0062 - acc: 0.9978 - val_loss: 0.0060 - val_acc: 0.9973
Epoch 4/20
 - 7s - loss: 0.0062 - acc: 0.9977 - val_loss: 0.0064 - val_acc: 0.9973
Epoch 5/20
 - 7s - loss: 0.0061 - acc: 0.9978 - val_loss: 0.0061 - val_acc: 0.9971
Epoch 6/20
 - 7s - loss: 0.0062 - acc: 0.9978 - val_loss: 0.0063 - val_acc: 0.9973
Epoch 7/20
 - 7s - loss: 0.0061 - acc: 0.9978 - val_loss: 0.0059 - val_acc: 0.9975
Epoch 8/20
 - 7s - loss: 0.0060 - acc: 0.9979 - val_loss: 0.0060 - val_acc: 0.9976
Epoch 9/20
 - 7s - loss: 0.0060 - acc: 0.9978 - val_loss: 0.0065 - val_acc: 0.9973
Epoch 10/20
 - 7s - loss: 0.0061 - acc: 0.9977 - val_loss: 0.0062 - val_acc: 0.9975
Epoch 11/20
 - 7s - loss: 0.0060 - acc: 0.9979 - val_loss: 0.0058 - val_acc: 0.9979
Epoch 12/20
 - 7s - loss: 0.0060 -

In [160]:
make_predictions(X,y, out=False, model=GRU_model)

Recall =  0.929786066922655 Precision =  0.9918080748976009
[[69562    14]
 [  128  1695]]


In [0]:
#make_predictions(X_dev,y_dev, out=True, model=GRU_model)

In [161]:
make_predictions(X_test, y_test, out=True, model=GRU_model, X_ini=test)

Recall =  0.5217391304347826 Precision =  0.6666666666666666
[[ 1 12]
 [22 24]]
0.0 [0.99998224] [-0.499 -0.472 -0.478 -0.465 -0.105 -0.304] [[-0.8279426555208718], [-0.7814679923192585], [-0.7917956952529502], [-0.7694190055632847], [-0.14975682954177363], [-0.49229231017588665]]
0.0 [0.99998146] [-0.499 -0.472 -0.478 -0.465 -0.105 -0.253] [[-0.8279426555208718], [-0.7814679923192585], [-0.7917956952529502], [-0.7694190055632847], [-0.14975682954177363], [-0.4045068352395059]]
0.0 [0.9999831] [-0.499 -0.472 -0.478 -0.465 -0.105 -0.231] [[-0.8279426555208718], [-0.7814679923192585], [-0.7917956952529502], [-0.7694190055632847], [-0.14975682954177363], [-0.3666385911493025]]
0.0 [0.9999709] [-0.499 -0.472 -0.478 -0.465 -0.048 -0.444] [[-0.8279426555208718], [-0.7814679923192585], [-0.7917956952529502], [-0.7694190055632847], [-0.051643651671701096], [-0.7332720452953632]]
0.0 [0.9999666] [-0.499 -0.472 -0.478 -0.465 -0.048 -0.394] [[-0.8279426555208718], [-0.7814679923192585], [-0.79179

In [162]:
make_predictions(X_test2, y_test2, out=True, model=GRU_model, X_ini=test2)

Recall =  0.8490566037735849 Precision =  0.703125
[[ 0 19]
 [ 8 45]]
0.0 [0.99983644] [-0.377 -0.231 -0.2   -0.155  0.027  0.208] [[-0.6179460292024708], [-0.3666385911493025], [-0.3132787926585613], [-0.2358210206558724], [0.07745263499944702], [0.3890050068324845]]
0.0 [0.9998486] [-0.377 -0.231 -0.2   -0.155  0.027  0.227] [[-0.6179460292024708], [-0.3666385911493025], [-0.3132787926585613], [-0.2358210206558724], [0.07745263499944702], [0.42170939945584196]]
0.0 [0.99985975] [-0.377 -0.231 -0.2   -0.155  0.027  0.242] [[-0.6179460292024708], [-0.3666385911493025], [-0.3132787926585613], [-0.2358210206558724], [0.07745263499944702], [0.4475286567900716]]
0.0 [0.99987173] [-0.377 -0.231 -0.2   -0.155  0.027  0.261] [[-0.6179460292024708], [-0.3666385911493025], [-0.3132787926585613], [-0.2358210206558724], [0.07745263499944702], [0.48023304941342915]]
0.0 [0.9998816] [-0.377 -0.231 -0.2   -0.155  0.027  0.279] [[-0.6179460292024708], [-0.3666385911493025], [-0.3132787926585613], [-0

In [0]:
#plot_loss(epochs, logplot_loss(epos.history.get('loss'), logs.history.get('val_loss'))

In [0]:
#acc_graph(epochs, logs.history.get('acc'), logs.history.get('val_acc'))

In [0]:
GRU_model.save_weights('GRU_sequence_weights.h5')

In [0]:
files.download('GRU_sequence_weights.h5')

In [0]:
LSTM_model = model((Tx,1),Ty)
#LSTM_model.summary()
LSTM_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
logs = LSTM_model.fit(X_train, y_train , epochs=epochs, batch_size=batch_size, shuffle=True,\
               validation_data=(X_dev, y_dev), verbose=0, class_weight=class_weights_train)

# Final fit NN on all train set

In [0]:
Tx=  6
Ty = 1
epochs = 250
batch_size = 32

In [0]:
LSTM_model = model((Tx,1),Ty)
#LSTM_model.summary()
LSTM_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
LSTM_model.fit(X, y , epochs=epochs, batch_size=batch_size, shuffle=True,\
               verbose=0, class_weight=class_weights)
LSTM_model.save_weights('sources/LSTM_sequence_weights.h5')

In [0]:
print(LSTM_model.summary())
test_preds = LSTM_model.predict(X_test)

In [0]:
for i in range(X_test.shape[0]):
    if test_preds[i] > 0.5:
        print(i)

In [0]:
LSTM_model = model((Tx,1),Ty)
LSTM_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

LSTM_model.load_weights('LSTM_sequence_weights.h5')

In [0]:
test_preds = LSTM_model.predict(X_test)

for i in range(X_test.shape[0]):
    if test_preds[i] > 0.5:
        print(i)